# Exploratory Data Analysis of the Greenwood Data

This Exploratory Data Analysis is being done to help with the spot check of certain columns to identify data that needs to be fixed.

We will follow the steps below:
- Load all the volumes data into one DataFrame
- Select the columns that we are going to work on
- Identify the unique values on each column with the counts
- Will select the burial records that contains the cases we want to fix
- Fix the records
- Generate a new json file with the fixes to update our interments index

## Initializing the Notebook

In [5]:
import pandas as pd
import os
import duckdb

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Initialize the sql database

In [6]:
%sql duckdb:///sqlite/db.duckdb

%sql SET GLOBAL pandas_analyze_sample=600000

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

load_volumes = False

## Loading all volumes

We are going to load all volumes json files into one single in memory structure called DataFrame, that will give us analytical capabilities to identify the values on each column and to fix, if needed.

In [7]:
if load_volumes:
  records = []
  for file in os.listdir('json'):
    records.append(pd.read_json(os.path.join('json',file)))

  df = pd.concat(records)

Now, we are going to import into duckdb, to be able to use SQL to do our validation.

In [8]:
if load_volumes:
  %sql drop table if exists interments;

  %sql create table interments as select * from df;

## Identify Columns to Work

Our first step to identify the columns we want to work with. We will do this by looking the columns in the search filters of the website.
The columns we identify there are:
- date of interment
- birthplace
- marital status
- age at death
- late residence
- place of death
- cause of death
- date of death
- undertaker
- burial registry
- lot number

Next step is to list the columns on our DataFrame.

In [9]:
if load_volumes:
  columns = df.columns.to_list()

  for idx, column in enumerate(columns):
    print(column,end=', ')
    if (idx+1) % 7 == 0:
        print('')

Now, we are going to map the columns we want.
- date of interment = interment_date_year_transcribed
- birthplace = birth_place_displayed
- marital status = marital_status
- age at death = age_years
- late residence = residence_place_city_display
- place of death = death_place_display
- cause of death = cause_of_death_display
- date of death = death_date_year_transcribed
- undertaker = undertaker_display
- burial registry = interment_id, registry_volume
- lot number = urial_location_lot_current


Now, we are going to work with SQL statements to discover issues with the data on those fields.

### Scratchpad

In [10]:
%%sql

select cause_of_death_display as cause_of_death, count(*) as records
from interments
group by cause_of_death_display
order by 1 
limit 10

cause_of_death,records
None,14530
& Bese,1
& Prolonged Labor,1
(1) Old Age (2) Diarrhoea,1
(Accidental) Mercury Bichloride,1
(Charity Hospital) Brights,1
(Heart Failure ) Typhoid Fever,1
(Non Contagious),1
(Post Operation) Exophthalmia Goitres,1
1827 Of The Brain,1


In [11]:
%%sql

update interments
set cause_of_death_display = ltrim(replace(cause_of_death_display, '"', ''))

Count
433848


In [12]:
%%sql

update interments
set cause_of_death_display = NULL
where cause_of_death_display = '' 
or cause_of_death_display = '-'
or cause_of_death_display = '- P' 

Count
0


In [13]:
%%sql

select cause_of_death_display, ltrim(regexp_replace(cause_of_death_display,'[0-9]','','g'))
from interments
where cause_of_death_display like '1%'

cause_of_death_display,"ltrim(regexp_replace(cause_of_death_display, '[0-9]', '', 'g'))"
1827 Of The Brain,Of The Brain
1863 Dysentery,Dysentery
1871 Scarlatina,Scarlatina
1878 Fatty Heart,Fatty Heart
1878 Asthenia Meningitis,Asthenia Meningitis
1879 Convulsions,Convulsions
1879 Peritonitis,Peritonitis
1879 Brights,Brights
1849 Consumption,Consumption


In [17]:
%%sql

select cause_of_death_display as cause_of_death, count(*) as records
from interments
group by cause_of_death_display
order by 1
limit 10

cause_of_death,records
None,14530
& Bese,1
& Prolonged Labor,1
(1) Old Age (2) Diarrhoea,1
(Accidental) Mercury Bichloride,1
(Charity Hospital) Brights,1
(Heart Failure ) Typhoid Fever,1
(Non Contagious),1
(Post Operation) Exophthalmia Goitres,1
1827 Of The Brain,1


In [19]:
%%sql 

select column_name
from information_schema.columns
where table_name = 'interments'
and column_name like 'a%'

column_name
age_years_transcribed
age_months_transcribed
age_days_transcribed
age_hours_transcribed
age_display
age_years
age_months
age_days
age_hours


In [46]:
%%sql

with raw_data as (
SELECT interment_id, registry_volume, registry_image, try_cast(age_years_transcribed as integer) as age_years, age_years_transcribed, age_display
FROM interments)
select * from raw_data
where age_years > 117

interment_id,registry_volume,registry_image,age_years,age_years_transcribed,age_display
406593,56.0,Volume 56_093,779,779,"779 years, 5 days"
9864.0,1.0,Volume 01_221,339,339,"339 years, 8 months, 13 days"
375371,52.0,Volume 52_042,788,788,"788 years, 5 months, 13 days"
384834,53.0,Volume 53_097,664,664,"664 years, 2 months, 8 days"
349562.0,49.0,Volume 49_026,809,809,"809 years, 1 month"
394535.0,55.0,Volume 55_041,862,862,"862 years, 19 months"
311572,43.0,Volume 43_056,789,789,789 years
311619,43.0,Volume 43_057,289,289,"289 years, 6 months"
